In [ ]:
from src.utils.config_loader import load_yaml_config
from src.tuning.param_grid import generate_param_grid
from pprint import pprint as pp

# Change below to the path of your config file
config = load_yaml_config("../../src/tuning/configs/modset1.yaml")
# Specify the model name you want to test params for (should match as labelled in the config)
param_grid = generate_param_grid("ARIMA", config)

pp(param_grid)

In [ ]:
# For testing only: this can be used to simplify the dictionary returned by `get_params()`
# so that the param grid has only one value for each parameter
def simplify_dict(d):
    for key, value in d.items():
        if isinstance(value, list):
            # Keep only the first element of the list (max length 1)
            d[key] = [value[0]]	# If you want to test other params, change this to a different index
    return d

In [ ]:
# To speed up testing we will only test one value for each parameter
param_grid = simplify_dict(param_grid)
param_grid

In [ ]:
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.split import ExpandingSlidingWindowSplitter
from sktime.performance_metrics.forecasting import MeanSquaredError
from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA  # Change this to the model you want to test

# Not the actual data we will use, just a placeholder for simple testing
y = load_airline()
fh = [1, 2, 3, 4, 5, 6]
cv = ExpandingSlidingWindowSplitter(
    fh=fh, initial_window=12, step_length=12, max_expanding_window_length=24 * 12
)

# Specify the forecaster you generated the param grid for
forecaster = ARIMA()

gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    # Simplify the dictionary so only one set of values are tested
    param_grid=param_grid,
    cv=cv,
    scoring=MeanSquaredError(square_root=True),
	# Raise errors so we can see what params are causing errors
    error_score="raise",
)
gscv.fit(y)
y_pred = gscv.predict(fh)

In [ ]:
y_pred

In [ ]:
gscv.best_params_